In [ ]:
#from bs4 import BeautifulSoup
#import json
#import optparse
import os, regex as re
import pandas as pd


#import libraries 

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import re 
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy as sp

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk import pos_tag

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2, SelectKBest

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.cross_validation import train_test_split, cross_val_score, cross_val_predict
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, cross_val_score

from sklearn import naive_bayes
from sklearn import svm, grid_search
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from mlens.ensemble import SuperLearner

from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report

from nltk.stem.porter import PorterStemmer

import imblearn
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler

In [ ]:
def read_data(filename):
    list1 = []
    reviews = []
    with open(filename, 'r+') as fr:
        print(fr)
        end_of_review = 0
        for line in fr:
            #print("is not empty")
            line = re.sub(r'[:][\d]', " ", str(line))
            if (re.search("#label#:negative", str(line))):
                line = re.sub("#label#:negative", " " ,str(line))
                end_of_review=1
            if (re.search("#label#:positive", str(line))):
                line = re.sub("#label#:positive", " " ,str(line))
                end_of_review=1    
            str1 = str(line)    
                #print("end-of review")
            if end_of_review == 1:    
                reviews.append(str1)
                end_of_review = 0
              
           
        return (reviews)
            #print(list1)
        
def convert_to_dataframe(listname):
    df1 = pd.DataFrame({'reviews':listname})
    return df1


def get_label_from_filename(filename, df):
    if re.search("positive", str(filename)):
        df["label"] = 1
    if  re.search("negative", str(filename)):
        df["label"] = 0 
        #pd.set_option('display.max_colwidth', -1)
    return df   


In [ ]:
# books_dataset
neg_reviews_list = read_data('Downloads/processed_acl/books/negative.review')
df1 = convert_to_dataframe(neg_reviews_list)
df1 =get_label_from_filename('Downloads/processed_acl/books/negative.review', df1)


pos_reviews_list = read_data('Downloads/processed_acl/books/positive.review')
df2 = convert_to_dataframe(pos_reviews_list)
df2 =get_label_from_filename('Downloads/processed_acl/books/positive.review', df2)


df_books = pd.concat([df1, df2], axis=0)

# dvd_dataset
neg_reviews_list2 = read_data('Downloads/processed_acl/dvd/negative.review')
df3 = convert_to_dataframe(neg_reviews_list2)
df3 =get_label_from_filename('Downloads/processed_acl/dvd/negative.review', df3)




pos_reviews_list2 = read_data('Downloads/processed_acl/dvd/positive.review')
df4 = convert_to_dataframe(pos_reviews_list2)
df4 =get_label_from_filename('Downloads/processed_acl/dvd/positive.review', df4)

df_dvd = pd.concat([df3, df4], axis=0)

# kitchen_dataset
neg_reviews_list3 = read_data('Downloads/processed_acl/kitchen/negative.review')
df5 = convert_to_dataframe(neg_reviews_list3)
df5 =get_label_from_filename('Downloads/processed_acl/kitchen/negative.review', df5)


pos_reviews_list3 = read_data('Downloads/processed_acl/kitchen/positive.review')
df6 = convert_to_dataframe(pos_reviews_list3)
df6 =get_label_from_filename('Downloads/processed_acl/kitchen/positive.review', df6)

df_kitchen = pd.concat([df5, df6], axis=0)

# electronics_dataset
neg_reviews_list4 = read_data('Downloads/processed_acl/electronics/negative.review')
df7 = convert_to_dataframe(neg_reviews_list4)
df7 =get_label_from_filename('Downloads/processed_acl/electronics/negative.review', df7)


pos_reviews_list4 = read_data('Downloads/processed_acl/electronics/positive.review')
df8 = convert_to_dataframe(pos_reviews_list4)
df8 =get_label_from_filename('Downloads/processed_acl/electronics/positive.review', df8)

df_electronics = pd.concat([df7, df8], axis=0)

In [ ]:

#adding column for number of words in review in original data frame
df_books['#words'] = df_books.reviews.apply(lambda x: len(str(x).split(' ')))
df_dvd['#words'] = df_dvd.reviews.apply(lambda x: len(str(x).split(' ')))
#e_df['#words'] = e_df.reviewText.apply(lambda x: len(str(x).split(' ')))
#k_df['#words'] = k_df.reviewText.apply(lambda x: len(str(x).split(' ')))
df_kitchen['#words'] = df_kitchen.reviews.apply(lambda x: len(str(x).split(' ')))
df_electronics['#words'] = df_electronics.apply(lambda x: len(str(x).split(' ')))

In [ ]:
#Shuffling the rows in all the datasets to make them randomly ordered
df_books.sample(frac=1)
df_books = df_books.sample(frac=1).reset_index(drop=True)
df_books["code"] = "books"

df_dvd.sample(frac=1)
df_dvd = df_dvd.sample(frac=1).reset_index(drop=True)
df_dvd["code"] = "dvd"

df_kitchen.sample(frac=1)
df_kitchen = df_kitchen.sample(frac=1).reset_index(drop=True)
df_kitchen["code"] = "kitchen"

df_electronics.sample(frac=1)
df_electronics = df_electronics.sample(frac=1).reset_index(drop=True)
df_electronics["code"] = "electronics"

# Preprocessing

In [ ]:
#Functions for preprocessing steps
stop = set(('i','im','ive', 'me','my','myself','we','our','ours','ourselves','you','youre','youve','youll','youd','your','yours','yourself','yourselves','he','him','his','himself','she','shes','her','hers','herself','it','its','itself','they','them','their','theirs','themselves','what','which','who','whom','this','that','thatll','these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and','but','if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after','above','below','to','from','up','down','in','out','on','off','over','under','again','further','then','once','here','there','when','where','why','how','all','any','both','each','few','more','most','other','some','such','only','own','same','so','than','too','very','s','t','can','will','just','should','shouldve','now','d','ll','m','o','re','ve','y','ma'))
sno = nltk.stem.SnowballStemmer('english')


def replace_url(df,col,rm1,rm2):
    return(df[col].str.replace(rm1,rm2))

def extract_emo(df, col, emo):
    return(df[col].str.extractall(emo).unstack().apply(lambda x:' '.join(x.dropna()), axis=1))

def replace_emo(df,col,emo1,emo2):
    return(df[col].str.replace(emo1,emo2))

def replace_punct(df, col, punct1, punct2):
    return(df[col].str.replace(punct1, punct2))

def lower_words(df,col):
    return(df[col].apply(lambda x: " ".join(x.lower() for x in x.split())))

def remove_stop(df,col):
    return(df[col].apply(lambda x: " ".join(x for x in x.split() if x not in stop)))

def tokenize(df,col):
    return(df.apply(lambda row: nltk.word_tokenize(row[col]), axis=1))

def word_count(df,col):
    return(df[col].apply(lambda x: len(str(x).split(' '))))

def stemming(df,col):
    return(df[col].apply(lambda x: " ".join([sno.stem(word) for word in x.split()])))


#Step1 Pre-Processing
sample_df['nohtml'] = replace_url(sample_df,'reviews','^http?:\/\/.*[\r\n]*','')
#sample_df['emoticons']=extract_emo(sample_df,'nohtml','(=\)|:\)|:-\)|:\(|:-\(|;\)|;-\)|:D|=D|:-D|=\(|:P|=P|:-P)')
#sample_df['nohtml'] = replace_emo(sample_df,'nohtml','(=\)|:\)|:-\)|:D|=D|:-D)',' happy ')
#sample_df['nohtml'] = replace_emo(sample_df,'nohtml','(=\(|:\(|:-\()',' sad ')
#sample_df['nohtml'] = replace_emo(sample_df,'nohtml','(;\)|;-\))',' wink ')
#sample_df['nohtml'] = replace_emo(sample_df,'nohtml','(:P|=P|:-P)',' playful ')
sample_df['nohtml'] = lower_words(sample_df,'nohtml')
sample_df['nohtml'] = replace_punct(sample_df, 'nohtml', '[^\w\s]',' ')
sample_df['nohtml'] = replace_punct(sample_df, 'nohtml', '_',' ')
sample_df['nohtml'] = replace_punct(sample_df, 'nohtml',r'\b(no|not|nt|dont|doesnt|didnt|cant|cannt|cannot|wouldnt|wont|couldnt|hasnt|havent|hadnt|shouldnt)\s+([a-z])',r'not \2')
sample_df['nohtml'] = remove_stop(sample_df,'nohtml')
#sample_df['nohtml'] = stemming(sample_df,'nohtml')
sample_df['tokenized'] = tokenize(sample_df,'nohtml')
sample_df['#token'] = word_count(sample_df,'tokenized')

In [ ]:
def find_num_nouns(df, col1, col2, col3):
    sample_df[col1]=df.tokenized.apply(lambda x : pos_tag(x))
    for  index, row in df.iterrows():
        row[col1]= [word for word,pos in row[col1] if pos.startswith('N')]
    #sample_df.set_value(index,'nouns',row["nouns"])   
        sample_df.set_value(index,col1,row[col1])
    sample_df[col2]=sample_df[col1].apply(lambda x: len(x)) 
    return (sample_df[col2].groupby(sample_df[col3]).sum())

# find the number of verbs per class
def find_num_verbs(df, col1, col2, col3):
    sample_df[col1]=df.tokenized.apply(lambda x : pos_tag(x))
    for  index, row in df.iterrows():
        row[col1]= [word for word,pos in row[col1] if pos.startswith('VB')]
    #sample_df.set_value(index,'nouns',row["nouns"])   
        sample_df.set_value(index,col1,row[col1])
    sample_df[col2]=sample_df[col1].apply(lambda x: len(x)) 
    return (sample_df[col2].groupby(sample_df[col3]).sum())

# find the number of adjectives per class
def find_num_adjective(df, col1, col2, col3):
    sample_df[col1]=df.tokenized.apply(lambda x : pos_tag(x))
    for  index, row in df.iterrows():
        row[col1]= [word for word,pos in row[col1] if pos.startswith('JJ')]
    #sample_df.set_value(index,'nouns',row["nouns"])   
        sample_df.set_value(index,col1,row[col1])
    sample_df[col2]=sample_df[col1].apply(lambda x: len(x)) 
    return (sample_df[col2].groupby(sample_df[col3]).sum())

# find the number of adverbs per class
def find_num_adverbs(df, col1, col2, col3):
    sample_df[col1]=df.tokenized.apply(lambda x : pos_tag(x))
    for  index, row in df.iterrows():
        row[col1]= [word for word,pos in row[col1] if pos.startswith('RB')]
    #sample_df.set_value(index,'nouns',row["nouns"])   
        sample_df.set_value(index,col1,row[col1])
    sample_df[col2]=sample_df[col1].apply(lambda x: len(x)) 
    return (sample_df[col2].groupby(sample_df[col3]).sum())

# find the number of total words per class expect the spected category
def find_num_others(df, col1, col2, col3):
    sample_df[col1]=df.tokenized.apply(lambda x : pos_tag(x))
    for  index, row in df.iterrows():
        row[col1]= [word for word,pos in row[col1]if not pos.startswith('N')]
    #sample_df.set_value(index,'nouns',row["nouns"])   
        sample_df.set_value(index,col1,row[col1])
    sample_df[col2]=sample_df[col1].apply(lambda x: len(x)) 
    return (sample_df[col2].groupby(sample_df[col3]).sum())

In [ ]:
find_num_verbs(sample_df, "pos_tagged2", "#verbs", "label")
find_num_adjective(sample_df, "pos_tagged3", "#adjectives", "label")
find_num_adverbs(sample_df, "pos_tagged4", "#adverbs", "label")
find_num_others(sample_df, "pos_tagged5", "#words", "label")

In [ ]:
###### 25
# chi-squared test with similar proportions
from scipy.stats import chi2_contingency
from scipy.stats import chi2
# contingency table
table = [[107099, 107515],
[151608, 142169]]
print(table)
stat, p, dof, expected = chi2_contingency(table)
print('dof=%d' % dof)
print(expected)
# interpret test-statistic
prob = 0.95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')
# interpret p-value
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')